In [1]:
import pandas as pd


pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)

In [2]:
filePath = r"D:\Rochester Institute of Technology\DSCI 644 Software Engineering for Data Science\Automated-Software-Refactoring-Commit-Classifier\dataset\ESWA20_manualclassification.csv"

In [3]:
df = pd.read_csv(filePath, encoding='ISO-8859-1')

df

,Commit Message,Category
0,\tNIO Reads writes are completed in the caller thered;\n\t100% compatible with old interface and functionality.\n\tEach connection has an instance of this object to\n\treceive data for a message in multiple OP_READ events.\n,functional
1,* temporary fix for non-shared JArray objects\n * added generic unittests to check jcc/lucene/solr/montysolr functionality\n * wrote unittest for newseman (it does a complete processing from python side)\n - now what remains is to call it from java\n - decide on proper behaviour for cases when tokens are added/grouped\n,bugfix
2,* Java:\n * Added RCON functionality for Source servers:\n * Added steamcondenser.steam.packets.rcon package\n * Added RCONAuthRequestPacket\n * Added RCONAuthResponsePacket\n * Added RCONExecRequestPacket\n * Added RCONExecResponsePacket\n * Added RCONPacket\n * Added RCONNoAuthException\n * Split SteamSocket into QuerySocket and RCONSocket\n * Stub functionality for GoldSrc servers\n * GoldSrc uses UDP and needs a special implementation\n * Added tests\n,functional
3,- Merge changes from Pull-Request #6 \t - Fix aggregation generation and cleanup dead code - Update dependencies versions,code smell
4,Bug 233643 - API builder performance bad for incremental build,external
...,...,...
1735,working on #339: introducing the getSpecialFeatureCollection Method as a more general way to get all the PrintTemplateFeatures. improve the numbering\n,functional
1736,working on #339: next step. i\n\nmove all the calculation and handling into the PrintTemplateFeature to handle\nmultiple templateprinting.\n\nThe creation is still very basic though\n,functional
1737,working on #369: generalized the adjustMapForPrintingTemplates and ensureVisibilityOfPrintingTemplates method to adjustMapForSpecialFeatureClasses and ensureVisibilityOfSpecialFeatures\n,functional
1738,working up test coverage; minor tweaks; removing dead code; fixing very minor bugs; adding tests,code smell


In [4]:
df['commit_message'] = df['Commit Message']
del df['Commit Message']

df['category'] = df['Category']
del df['Category']

df

,commit_message,category
0,\tNIO Reads writes are completed in the caller thered;\n\t100% compatible with old interface and functionality.\n\tEach connection has an instance of this object to\n\treceive data for a message in multiple OP_READ events.\n,functional
1,* temporary fix for non-shared JArray objects\n * added generic unittests to check jcc/lucene/solr/montysolr functionality\n * wrote unittest for newseman (it does a complete processing from python side)\n - now what remains is to call it from java\n - decide on proper behaviour for cases when tokens are added/grouped\n,bugfix
2,* Java:\n * Added RCON functionality for Source servers:\n * Added steamcondenser.steam.packets.rcon package\n * Added RCONAuthRequestPacket\n * Added RCONAuthResponsePacket\n * Added RCONExecRequestPacket\n * Added RCONExecResponsePacket\n * Added RCONPacket\n * Added RCONNoAuthException\n * Split SteamSocket into QuerySocket and RCONSocket\n * Stub functionality for GoldSrc servers\n * GoldSrc uses UDP and needs a special implementation\n * Added tests\n,functional
3,- Merge changes from Pull-Request #6 \t - Fix aggregation generation and cleanup dead code - Update dependencies versions,code smell
4,Bug 233643 - API builder performance bad for incremental build,external
...,...,...
1735,working on #339: introducing the getSpecialFeatureCollection Method as a more general way to get all the PrintTemplateFeatures. improve the numbering\n,functional
1736,working on #339: next step. i\n\nmove all the calculation and handling into the PrintTemplateFeature to handle\nmultiple templateprinting.\n\nThe creation is still very basic though\n,functional
1737,working on #369: generalized the adjustMapForPrintingTemplates and ensureVisibilityOfPrintingTemplates method to adjustMapForSpecialFeatureClasses and ensureVisibilityOfSpecialFeatures\n,functional
1738,working up test coverage; minor tweaks; removing dead code; fixing very minor bugs; adding tests,code smell


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1740 entries, 0 to 1739
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   commit_message  1740 non-null   object
 1   category        1740 non-null   object
dtypes: object(2)
memory usage: 27.3+ KB


In [6]:
df.category.unique()

array(['functional', 'bugfix', 'code smell', 'external', 'internal'],
      dtype=object)

In [7]:
df.nunique()

commit_message    1716
category             5
dtype: int64

In [8]:
df = df.drop_duplicates()

In [9]:
df.duplicated().sum()

0

In [10]:
df.isnull().sum()

commit_message    0
category          0
dtype: int64

In [11]:
df.isna().sum()

commit_message    0
category          0
dtype: int64

In [12]:

only_spaces_count = df['commit_message'].apply(lambda x: x.isspace() if pd.notnull(x) else False).sum()

only_spaces_count


0

In [13]:
df.count()

commit_message    1721
category          1721
dtype: int64

In [14]:
print(df.iloc[797])

commit_message    JBRULES-1450 De-couple drools-core from the java dialect's PackageCompilationData -Initial de-coupling attempt -also updated to mvel 2.0 alpha 2
category                                                                                                                                                  internal
Name: 803, dtype: object


### Text Pre-processing

In [15]:
# from pycontractions import Contractions
# import re
from contractions import fix
# from gensim.utils import contractions

In [16]:
# Find commit messages containing "I'm"
contains_im = df[df['commit_message'].str.contains("I'll", na=False)]

print("Before fixing contractions:")
print(contains_im['commit_message'].head(10))

Before fixing contractions:
729                            Implement my old patch for issue #366, with some tweaks.  This does the\nfollowing:\n* Fixes @Provides injection so that parameters are checked for nullability.\nBy default this will error.  The flag is named:\nguice_check_nullable_provides_params and can be set to ERROR, WARNING or IGNORE.\n* Adds InjectionPoint.forMethod to build an InjectionPoint off an arbitrary\nmethod.\n* Adds Binder.getProvider(Dependency) to a get a Provider for a given\ndependency (with all its nullability & injection points maintained).\n* Update ProviderLookup to accept a Dependency in addition to a Key.\n\nThis is in preparation for two things:\n1) Allowing multibindings/mapbindings/optionalbindings to be specified as\nannotations on methods in a module.\n2) Adding a dagger compatibility module.\n... the general idea will be that I'll also add a hook into\nProvidesMethodModule somehow to look at arbitrary other annotations and let\nfolks process the

In [17]:
# # Function to fix contractions in a string
# def expand_contractions(text):
#     return fix(text)


# # Identify and print rows with contractions
# # We're looking for apostrophes as a simple heuristic for finding contractions
# contraction_rows = df[df['commit_message'].str.contains("'")]


# # Apply the function to your data to fix contractions
# # Assuming 'commit_message' is the column with text data
# df['expanded_commit_message'] = df['commit_message'].apply(expand_contractions)

# # Display the first 10 rows with contractions before and after expansion
# print(contraction_rows[['commit_message', 'expanded_commit_message']].head(10))


# # Find commit messages containing "I'm"
# contains_im = df[df['commit_message'].str.contains("I'm", na=False)]

# # Print messages before fixing contractions
# print("Before fixing contractions:")
# print(contains_im['commit_message'].head(10))

# Define a function to fix contractions in a string
def expand_contractions(text):
    return fix(text)

# Apply the function to expand contractions for the whole column (if needed later)
df['commit_message'] = df['commit_message'].apply(expand_contractions)



# # Fix contractions and print messages after fixing
# contains_im['commit_message'].head(10).apply(lambda x: print(f"After fixing contractions:\n{expand_contractions(x)}\n"))


C:\Users\onkar\AppData\Local\Temp\ipykernel_35572\3244773756.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['commit_message'] = df['commit_message'].apply(expand_contractions)


In [18]:
print(df.iloc[1325])

commit_message    [#3481] Help link on the bottom of the page not working [#3527] Lazy init exception when viewing Cherry Pick Request [#3520] Show compound name column in the study annotation viewer [#3528] Add a total Lab Cherry Picks column to the CherryPickRequestSearchResults [#3529] Server crash on export of library images to xls format [#3511] "Medicinal Chemist User" Facility Usage Role should be changed to "Medicinal Chemistry User" [#3531] Facility User Role "QPRC User" is incorrect [#3510] Increase the size of the confidentiality notice on the login page [#3478] Modifications for #3478 - PropertyPathDataFetcher based Search results sort null values higher than non-null values (in PostgreSQL)
category                                                                                                                                                                                                                                                                                        

In [19]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\onkar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
import re

In [21]:
import re

def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove single-character tokens (mostly punctuation)
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    # Remove single characters from the start
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)
    # Substitute multiple spaces with single space
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    # Remove prefixed 'b'
    text = re.sub(r'^b\s+', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text




In [22]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


stop_words = set(stopwords.words('english'))

custom_stop_words = ['git', 'svn', 'refactor', 'svn', 'gitsvnid', 'signedoffby', 'reviewedon', 'testedby', 'us', 'id', 'changeid', 'lot', 'small', 'thing', 'way']  # Add more custom words here
stop_words.update(custom_stop_words)

def remove_stopwords(text):
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.lower() not in stop_words]
    return ' '.join(filtered_text)


In [23]:
# Find the top 5 rows with the maximum lengths in 'col1' (sorted by descending length)
top_5_max_lengths = df['commit_message'].str.len().sort_values(ascending=False).head(5)

# Print the top 5 rows with corresponding lengths
for index, length in top_5_max_lengths.items():
  print(f"\nRow Index: {index}")
  print(df.loc[index])
  print(f"Length: {length}")




Row Index: 400
commit_message    Changes based on Keith's review.  A few tweaks to EdgeSet to help tracking edge\nremoval\n\n\tmodified:   src/edu/ucla/sspace/common/Similarity.java\n\n- Updated to use VectorMath.dotProduct for the Tanimoto coefficient\n\n\tdeleted:    src/edu/ucla/sspace/common/WordComparator.java\n\n- Moved to SimpleNearestNeighborFinder\n\n\tmodified:   src/edu/ucla/sspace/dependency/SimpleDependencyPath.java\n\n- Removed println\n\n\tmodified:   src/edu/ucla/sspace/graph/AbstractGraph.java\n\n- Added missing implementation to Subgraph class so now all the unit tests pass\n\n\tmodified:   src/edu/ucla/sspace/graph/DirectedMultigraph.java\n\n- Added missing implementation to Subgraph class so now all the unit tests pass\n\n- Fixed bug for reporting the correct edge types after removal\n\n- Removed dead code\n\n\tmodified:   src/edu/ucla/sspace/graph/EdgeSet.java\n\n- Updated so that disconnect() now returns the number of edges that were removed\n\n\tmodified:   src/

In [24]:
df['commit_message_clean'] = df['commit_message'].apply(clean_text)
df['commit_message_clean'] = df['commit_message_clean'].apply(remove_stopwords)


C:\Users\onkar\AppData\Local\Temp\ipykernel_35572\1127110524.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['commit_message_clean'] = df['commit_message'].apply(clean_text)
C:\Users\onkar\AppData\Local\Temp\ipykernel_35572\1127110524.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['commit_message_clean'] = df['commit_message_clean'].apply(remove_stopwords)


In [25]:
top_5_max_lengths = df['commit_message_clean'].str.len().sort_values(ascending=False).head(5)


for index, length in top_5_max_lengths.items():
  print(f"\nRow Index: {index}")
  print(df.loc[index])
  print(f"Length: {length}")




Row Index: 400
commit_message          Changes based on Keith's review.  A few tweaks to EdgeSet to help tracking edge\nremoval\n\n\tmodified:   src/edu/ucla/sspace/common/Similarity.java\n\n- Updated to use VectorMath.dotProduct for the Tanimoto coefficient\n\n\tdeleted:    src/edu/ucla/sspace/common/WordComparator.java\n\n- Moved to SimpleNearestNeighborFinder\n\n\tmodified:   src/edu/ucla/sspace/dependency/SimpleDependencyPath.java\n\n- Removed println\n\n\tmodified:   src/edu/ucla/sspace/graph/AbstractGraph.java\n\n- Added missing implementation to Subgraph class so now all the unit tests pass\n\n\tmodified:   src/edu/ucla/sspace/graph/DirectedMultigraph.java\n\n- Added missing implementation to Subgraph class so now all the unit tests pass\n\n- Fixed bug for reporting the correct edge types after removal\n\n- Removed dead code\n\n\tmodified:   src/edu/ucla/sspace/graph/EdgeSet.java\n\n- Updated so that disconnect() now returns the number of edges that were removed\n\n\tmodified: 

In [26]:
del df['commit_message']

df

,category,commit_message_clean
0,functional,NIO Reads writes completed caller would compatible old interface functionality connection instance object receive data message multiple OPREAD events
1,bugfix,temporary fix nonshared JArray objects added generic unittests check jcclucenesolrmontysolr functionality wrote unittest newseman complete processing python side remains call java decide proper behaviour cases tokens addedgrouped
2,functional,Java Added RCON functionality Source servers Added steamcondensersteampacketsrcon package Added RCONAuthRequestPacket Added RCONAuthResponsePacket Added RCONExecRequestPacket Added RCONExecResponsePacket Added RCONPacket Added RCONNoAuthException Split SteamSocket QuerySocket RCONSocket Stub functionality GoldSrc servers GoldSrc uses UDP needs special implementation Added tests
3,code smell,Merge changes PullRequest Fix aggregation generation cleanup dead code Update dependencies versions
4,external,Bug API builder performance bad incremental build
...,...,...
1735,functional,working introducing getSpecialFeatureCollection Method general get PrintTemplateFeatures improve numbering
1736,functional,working next step move calculation handling PrintTemplateFeature handle multiple templateprinting creation still basic though
1737,functional,working generalized adjustMapForPrintingTemplates ensureVisibilityOfPrintingTemplates method adjustMapForSpecialFeatureClasses ensureVisibilityOfSpecialFeatures
1738,code smell,working test coverage minor tweaks removing dead code fixing minor bugs adding tests


In [27]:
from nltk.tokenize import word_tokenize

df['tokens'] = df['commit_message_clean'].apply(word_tokenize)


C:\Users\onkar\AppData\Local\Temp\ipykernel_35572\653882617.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens'] = df['commit_message_clean'].apply(word_tokenize)


In [28]:
from nltk import pos_tag

df['pos_tags'] = df['tokens'].apply(pos_tag)


C:\Users\onkar\AppData\Local\Temp\ipykernel_35572\3072077768.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pos_tags'] = df['tokens'].apply(pos_tag)


In [29]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

# Function to map NLTK position tags
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun if unknown

# Lemmatize a sentence with the appropriate POS tag
def lemmatize_with_pos(sentence):
    return [lemmatizer.lemmatize(word, get_wordnet_pos(pos_tag)) for word, pos_tag in sentence]

# Apply the lemmatization function to the pos tags
df['lemmatized'] = df['pos_tags'].apply(lemmatize_with_pos)


C:\Users\onkar\AppData\Local\Temp\ipykernel_35572\2141383612.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lemmatized'] = df['pos_tags'].apply(lemmatize_with_pos)


In [30]:
df['lemmatized_text'] = df['lemmatized'].apply(' '.join)


C:\Users\onkar\AppData\Local\Temp\ipykernel_35572\2955464519.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lemmatized_text'] = df['lemmatized'].apply(' '.join)


In [31]:
# del df['commit_message_clean', 'tokens', 'pos_tags', 'lemmatized']

df = df[['lemmatized_text', 'category']]

df

,lemmatized_text,category
0,NIO Reads write complete caller would compatible old interface functionality connection instance object receive data message multiple OPREAD event,functional
1,temporary fix nonshared JArray object add generic unittests check jcclucenesolrmontysolr functionality write unittest newseman complete processing python side remain call java decide proper behaviour case token addedgrouped,bugfix
2,Java Added RCON functionality Source server Added steamcondensersteampacketsrcon package Added RCONAuthRequestPacket Added RCONAuthResponsePacket Added RCONExecRequestPacket Added RCONExecResponsePacket Added RCONPacket Added RCONNoAuthException Split SteamSocket QuerySocket RCONSocket Stub functionality GoldSrc server GoldSrc use UDP need special implementation Added test,functional
3,Merge change PullRequest Fix aggregation generation cleanup dead code Update dependency version,code smell
4,Bug API builder performance bad incremental build,external
...,...,...
1735,work introduce getSpecialFeatureCollection Method general get PrintTemplateFeatures improve numbering,functional
1736,work next step move calculation handle PrintTemplateFeature handle multiple templateprinting creation still basic though,functional
1737,work generalize adjustMapForPrintingTemplates ensureVisibilityOfPrintingTemplates method adjustMapForSpecialFeatureClasses ensureVisibilityOfSpecialFeatures,functional
1738,work test coverage minor tweak remove dead code fix minor bug add test,code smell


In [32]:

top_5_max_lengths = df['lemmatized_text'].str.len().sort_values(ascending=False).head(5)


for index, length in top_5_max_lengths.items():
  print(f"\nRow Index: {index}")
  print(df.loc[index])
  print(f"Length: {length}")




Row Index: 400
lemmatized_text    Changes base Keiths review tweaks EdgeSet help track edge removal modify srceduuclasspacecommonSimilarityjava Updated use VectorMathdotProduct Tanimoto coefficient delete srceduuclasspacecommonWordComparatorjava Moved SimpleNearestNeighborFinder modify srceduuclasspacedependencySimpleDependencyPathjava Removed println modify srceduuclasspacegraphAbstractGraphjava Added miss implementation Subgraph class unit test pass modify srceduuclasspacegraphDirectedMultigraphjava Added miss implementation Subgraph class unit test pass Fixed bug reporting correct edge type removal Removed dead code modify srceduuclasspacegraphEdgeSetjava Updated disconnect return number edge remove modified srceduuclasspacegraphGenericEdgeSetjava modify srceduuclasspacegraphSparseDirectedEdgeSetjava modify srceduuclasspacegraphSparseDirectedTypedEdgeSetjava modify srceduuclasspacegraphSparseTypedEdgeSetjava modify srceduuclasspacegraphSparseUndirectedEdgeSetjava modify srceduuclas

In [33]:
df.duplicated().sum()

10

In [34]:
df = df.drop_duplicates()

In [35]:
df.duplicated().sum()

0

In [36]:
df.isnull().count()

lemmatized_text    1711
category           1711
dtype: int64

In [37]:
# For rows
df = df.dropna(how='all')



In [38]:
df.count()

lemmatized_text    1711
category           1711
dtype: int64

In [39]:
df['lemmatized_text'].duplicated().sum()


6

In [40]:

df1 = df[df['lemmatized_text'].duplicated(keep=False)]

df1

,lemmatized_text,category
508,Enhanced tool palette composition functionality Added Morph ActivityGateway feature,internal
509,Enhanced tool palette composition functionality Added Morph ActivityGateway feature,external
577,Fix performance regresssion introduce use InetSocketAddress getHostName Patch provide Scott Harrington improve upon Kris Jurka,bugfix
578,Fix performance regresssion introduce use InetSocketAddress getHostName Patch provide Scott Harrington improve upon Kris Jurka,external
796,Issue size test compatibility image refactored vcloud config logic update support late bluelock,internal
797,Issue size test compatibility image refactored vcloud config logic update support late bluelock,external
862,Make bitcoinSerialize return copy default provide unsafeBitcoinSerialize method high performance application willing deal extra API complexity,internal
863,Make bitcoinSerialize return copy default provide unsafeBitcoinSerialize method high performance application willing deal extra API complexity,external
1241,StewartSingularValueDecomposition monolithic code ugly switch statement multiple method,internal
1242,StewartSingularValueDecomposition monolithic code ugly switch statement multiple method,code smell


In [41]:
# # Identify non-duplicated rows based on 'lemmatized_text'
# non_duplicated_rows = df.duplicated(subset='lemmatized_text', keep=False)

# # Keep only non-duplicated rows in the DataFrame
# df = df[~non_duplicated_rows]

# df


In [42]:
df.count()

lemmatized_text    1711
category           1711
dtype: int64

In [43]:
# Define a function that checks if a cell is only space characters
def is_only_spaces(x):
    return isinstance(x, str) and x.strip() == ''

# Apply the function to each element of the DataFrame
spaces_mask = df.applymap(is_only_spaces)

# all() to check if all values in each column are True (only spaces)
columns_only_spaces = spaces_mask.all()

print(columns_only_spaces)

columns_with_only_spaces = columns_only_spaces[columns_only_spaces].index.tolist()
print(columns_with_only_spaces)


lemmatized_text    False
category           False
dtype: bool
[]


C:\Users\onkar\AppData\Local\Temp\ipykernel_35572\370267273.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  spaces_mask = df.applymap(is_only_spaces)


In [44]:
df.count()

lemmatized_text    1711
category           1711
dtype: int64

### Training/Testing Split

In [45]:
from sklearn.model_selection import train_test_split

X = df['lemmatized_text']  
y = df['category']    


x_train, x_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,       # 25% of the data will be allocated to the test set
    stratify=y,           # Stratify based on the labels to maintain distribution
    random_state=42       
)


In [46]:
x_train

966     Performance enhancement use set define filter add module path mbean operation add additional logging message use exact match filter reduce resize overhead path map fix bug quote glob escape
1207                                                      Serialize session internal field dedicate byte array Introduced mockito test seem nice jmock providing require feature mock class interface
605                                                                           Fixed performance bug output stream outputwriters properly close affected performance render raw Strings raw byte array
1560                                                                                                                           Removing new findResources method since would huge performance problem
476                                                                                                                                    Delete dead code use anymore automatically provide DOM support
          

In [47]:
category_counts = y_test.value_counts()

print(category_counts)


category
functional    87
bugfix        87
internal      86
external      86
code smell    82
Name: count, dtype: int64


In [48]:
category_counts = y_train.value_counts()
print(category_counts)

category
bugfix        261
functional    259
external      259
internal      257
code smell    247
Name: count, dtype: int64


In [49]:
y_train.count()

1283

In [50]:
x_train_path = r"D:\Rochester Institute of Technology\DSCI 644 Software Engineering for Data Science\Automated-Software-Refactoring-Commit-Classifier\dataset\x_train.csv"

x_train.to_csv(x_train_path, index=False)

In [51]:
y_train_path = r"D:\Rochester Institute of Technology\DSCI 644 Software Engineering for Data Science\Automated-Software-Refactoring-Commit-Classifier\dataset\y_train.csv"

y_train.to_csv(y_train_path, index=False)

In [52]:
x_test_path = r"D:\Rochester Institute of Technology\DSCI 644 Software Engineering for Data Science\Automated-Software-Refactoring-Commit-Classifier\dataset\x_test.csv"

x_test.to_csv(x_test_path, index=False)

In [53]:
y_test_path = r"D:\Rochester Institute of Technology\DSCI 644 Software Engineering for Data Science\Automated-Software-Refactoring-Commit-Classifier\dataset\y_test.csv"

y_test.to_csv(y_test_path, index=False)

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [55]:
# Initialize TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=10000)

# Fit and transform the training data
x_train_tfidf = vectorizer.fit_transform(x_train)

# Only transform the testing data
x_test_tfidf = vectorizer.transform(x_test)

In [56]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.metrics import accuracy_score

# # Initialize classifiers
# classifiers = {
#     "Random Forest": RandomForestClassifier(),
#     "Logistic Regression": LogisticRegression(),
#     "Multinomial Naive Bayes": MultinomialNB(),
#     "K-Nearest Neighbors": KNeighborsClassifier(),
#     "Support Vector Classification": SVC(),
#     "Decision Tree": DecisionTreeClassifier()
# }

# # Dictionary to store the accuracy of each classifier
# accuracy_results = {}

# # Train and evaluate each classifier
# for classifier_name, classifier in classifiers.items():
#     # Train the classifier
#     classifier.fit(x_train_tfidf, y_train)
    
#     # Predict on the test set
#     y_pred = classifier.predict(x_test_tfidf)
    
#     # Evaluate the accuracy
#     accuracy = accuracy_score(y_test, y_pred)
#     accuracy_results[classifier_name] = accuracy
#     print(f"{classifier_name} Accuracy: {accuracy}")


In [57]:
# k-fold

In [58]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# # Initialize classifiers with specified hyperparameters
# classifiers = {
#     "Random Forest": RandomForestClassifier(max_depth=70, n_estimators=58, criterion='gini', bootstrap=False),
#     "Support Vector Classification": SVC(gamma=1.9, kernel='linear', C=1.0),
#     "Decision Tree": DecisionTreeClassifier(max_depth=75, criterion='gini'),
#     "Logistic Regression": LogisticRegression(penalty='l2', C=1.0, solver='liblinear'),
#     "Multinomial Naive Bayes": MultinomialNB(alpha=0.63),
#     "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=69, weights='uniform')
# }

# # Train and evaluate each classifier
# for classifier_name, classifier in classifiers.items():
#     # Train the classifier
#     classifier.fit(x_train_tfidf, y_train)
    
#     # Predict on the test set
#     y_pred = classifier.predict(x_test_tfidf)
    
#     # Calculate accuracy
#     accuracy = accuracy_score(y_test, y_pred)
#     print(f"{classifier_name} Accuracy: {accuracy}")


In [59]:
from sklearn.metrics import classification_report

classifiers = {
    "Random Forest": RandomForestClassifier(max_depth=78, n_estimators=500, criterion='gini', bootstrap=False),
    "Support Vector Classification": SVC(gamma='scale', kernel='linear', C=1.99),
    "Decision Tree": DecisionTreeClassifier(max_depth=75, criterion='gini'),
    "Logistic Regression": LogisticRegression(penalty='l1', C=1.0, solver='liblinear'),
    "Multinomial Naive Bayes": MultinomialNB(alpha=2.63),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=69, weights='uniform')
}

evaluation_reports = {}

# Train, predict, and evaluate each classifier separately
for classifier_name, classifier in classifiers.items():
    print(f"Training {classifier_name}...")
    
    # Train the classifier
    classifier.fit(x_train_tfidf, y_train)
    
    # Predict on the test set
    y_pred = classifier.predict(x_test_tfidf)
    
    # Generate a classification report
    report = classification_report(y_test, y_pred, output_dict=True)
    
    # Print accuracy and store the report for further analysis
    print(f"{classifier_name} Accuracy: {report['accuracy']}")
    evaluation_reports[classifier_name] = report
    
    # Detailed report
    print(f"{classifier_name} Classification Report:")
    print(classification_report(y_test, y_pred))
    


Training Random Forest...
Random Forest Accuracy: 0.8341121495327103
Random Forest Classification Report:
              precision    recall  f1-score   support

      bugfix       0.84      0.78      0.81        87
  code smell       0.95      0.99      0.97        82
    external       0.72      0.83      0.77        86
  functional       0.79      0.71      0.75        87
    internal       0.88      0.87      0.88        86

    accuracy                           0.83       428
   macro avg       0.84      0.84      0.84       428
weighted avg       0.84      0.83      0.83       428

Training Support Vector Classification...
Support Vector Classification Accuracy: 0.8060747663551402
Support Vector Classification Classification Report:
              precision    recall  f1-score   support

      bugfix       0.72      0.67      0.69        87
  code smell       0.90      0.93      0.92        82
    external       0.76      0.77      0.76        86
  functional       0.73      0.83 

In [60]:
    # def fit(self, X, y):
        
    #     X_preprocessed = preprocess(X.copy())
        
    #     # Separating text data for TF-IDF transformation
    #     text_data = X_preprocessed.pop('combined_text')
        
    #     text_features = self.tfidf_vectorizer.fit_transform(text_data)
        
    #     # Combining text features with other features
    #     X_combined = np.hstack((text_features.toarray(), X_preprocessed.values))
        
    #     self.classifier.fit(X_combined, y)
        
    #     # # Define a broad range of parameters for RandomizedSearchCV
    #     # rf_random_params = {
    #     #     'n_estimators': np.arange(100, 1001, 100),
    #     #     'max_depth': np.arange(10, 101, 10),
    #     #     'min_samples_split': np.arange(2, 11, 1),
    #     #     'criterion': ['gini', 'entropy']
    #     # }
        
    #     # # Randomized Search with Cross-Validation
    #     # self.rfc = RandomForestClassifier(class_weight="balanced", random_state=42)
    #     # random_search = RandomizedSearchCV(self.rfc, rf_random_params, n_iter=100, cv=5, scoring='f1', n_jobs=-1, random_state=42)
    #     # random_search.fit(X_combined, y)
    #     # print("Best parameters from RandomizedSearch: ", random_search.best_params_)

    #     # # Refine search with GridSearchCV around the best parameters found
    #     # best_params = random_search.best_params_
    #     # rf_grid_params = {
    #     #     'n_estimators': [best_params['n_estimators'] - 50, best_params['n_estimators'], best_params['n_estimators'] + 50],
    #     #     'max_depth': [best_params['max_depth'] - 10, best_params['max_depth'], best_params['max_depth'] + 10],
    #     #     'min_samples_split': [best_params['min_samples_split'] - 1, best_params['min_samples_split'], best_params['min_samples_split'] + 1],
    #     #     'criterion': [best_params['criterion']]
    #     # }
    #     # self.rscv = GridSearchCV(self.rfc, rf_grid_params, cv=5, scoring='f1', n_jobs=-1)
    #     # self.rscv.fit(X_combined, y)
    #     # print("Refined best parameters from GridSearchCV: ", self.rscv.best_params_)

In [69]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Assuming x_train_tfidf and y_train are already defined as your training dataset and labels

# Initialize RandomForestClassifier
rf_classifier = RandomForestClassifier(class_weight="balanced", random_state=42)

# Define the parameter space for RandomizedSearchCV
random_grid = {
    'n_estimators': np.arange(100, 1001, 100),  # Number of trees in the forest
    'max_depth': np.arange(10, 101, 10),  # Maximum depth of the tree
    'bootstrap': [True, False],  # Method of selecting samples for training each tree
    'criterion': ['gini', 'entropy']  # The function to measure the quality of a split
}

# Random search of parameters, using 3 fold cross validation,
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=rf_classifier, param_distributions=random_grid, n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)

# Fit the random search model
rf_random.fit(x_train_tfidf, y_train)

# Output the best parameters from RandomizedSearchCV
print("Best parameters found by RandomizedSearchCV:")
print(rf_random.best_params_)

# You can now take the best parameters from the random search and use them to
# create a more focused search with GridSearchCV (if necessary). This might involve
# narrower ranges of parameters or specific combinations that you want to test exhaustively.

# Refine search with GridSearchCV around the best parameters found
best_params = rf_random.best_params_
rf_grid_params = {
            'n_estimators': [best_params['n_estimators'] - 50, best_params['n_estimators'], best_params['n_estimators'] + 50],
            'max_depth': [best_params['max_depth'] - 10, best_params['max_depth'], best_params['max_depth'] + 10],            
            'criterion': [best_params['criterion']],
            'bootstrap': [best_params['bootstrap']]
}
rscv = GridSearchCV(rf_classifier, rf_grid_params, cv=5, scoring='f1', n_jobs=-1)
rscv.fit(x_train_tfidf, y_train)
print("Refined best parameters from GridSearchCV: ", rscv.best_params_)


# {'n_estimators': 1000, 'min_samples_split': 8, 'max_depth': 90, 'criterion': 'gini', 'bootstrap': False}
# Refined best parameters from GridSearchCV:  {'criterion': 'gini', 'max_depth': 80, 'min_samples_split': 7, 'n_estimators': 950}

# {'n_estimators': 200, 'max_depth': 40, 'criterion': 'gini', 'bootstrap': False}
# Refined best parameters from GridSearchCV:  {'bootstrap': False, 'criterion': 'gini', 'max_depth': 30, 'n_estimators': 150}

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best parameters found by RandomizedSearchCV:
{'n_estimators': 200, 'max_depth': 40, 'criterion': 'gini', 'bootstrap': False}


d:\Rochester Institute of Technology\DSCI 644 Software Engineering for Data Science\Automated-Software-Refactoring-Commit-Classifier\myenv\Lib\site-packages\sklearn\model_selection\_search.py:1051: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Refined best parameters from GridSearchCV:  {'bootstrap': False, 'criterion': 'gini', 'max_depth': 30, 'n_estimators': 150}


In [72]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.svm import SVC
import numpy as np

# Assuming x_train_tfidf and y_train are already defined

# Initialize the SVC model
svc = SVC(random_state=42)

# Define a parameter space for RandomizedSearchCV
param_distributions = {
    'C': np.logspace(-4, 4, 20),  # Regularization parameter
    'gamma': ['scale', 'auto'] + list(np.logspace(-9, 3, 13)),  # Kernel coefficient for 'rbf', 'poly' and 'sigmoid'
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid']  # Specifies the kernel type to be used in the algorithm
}

# Random search of parameters, using 3 fold cross validation,
# search across a wide range of combinations, and use all available cores
svm_random = RandomizedSearchCV(estimator=svc, param_distributions=param_distributions, n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)

# Fit the random search model
svm_random.fit(x_train_tfidf, y_train)

# Output the best parameters from RandomizedSearchCV
print("Best parameters found by RandomizedSearchCV for SVM:")
print(svm_random.best_params_)


# Extract the best parameters found by RandomizedSearchCV
best_params = svm_random.best_params_

# Create a parameter grid focused around the best parameters found
param_grid = {
    'C': [best_params['C'] * 0.5, best_params['C'], best_params['C'] * 2],
    'gamma': [best_params['gamma']] if best_params['gamma'] in ('scale', 'auto') else [best_params['gamma'] * 0.5, best_params['gamma'], best_params['gamma'] * 2],
    'kernel': [best_params['kernel']]
}

# Create a GridSearchCV for a more focused search
svm_grid = GridSearchCV(estimator=SVC(random_state=42), param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)

# Fit GridSearchCV
svm_grid.fit(x_train_tfidf, y_train)

# Output the best parameters from GridSearchCV
print("Refined best parameters from GridSearchCV for SVM:")
print(svm_grid.best_params_)


from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Assuming x_train_tfidf and y_train are already defined as your training dataset and labels

# Initialize RandomForestClassifier
rf_classifier = RandomForestClassifier(class_weight="balanced", random_state=42)

# Define the parameter space for RandomizedSearchCV
random_grid = {
    'n_estimators': np.arange(100, 1001, 100),  # Number of trees in the forest
    'max_depth': np.arange(10, 101, 10),  # Maximum depth of the tree
    'bootstrap': [True, False],  # Method of selecting samples for training each tree
    'criterion': ['gini', 'entropy']  # The function to measure the quality of a split
}

# Random search of parameters, using 3 fold cross validation,
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=rf_classifier, param_distributions=random_grid, n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)

# Fit the random search model
rf_random.fit(x_train_tfidf, y_train)

# Output the best parameters from RandomizedSearchCV
print("Best parameters found by RandomizedSearchCV:")
print(rf_random.best_params_)

# You can now take the best parameters from the random search and use them to
# create a more focused search with GridSearchCV (if necessary). This might involve
# narrower ranges of parameters or specific combinations that you want to test exhaustively.

# Refine search with GridSearchCV around the best parameters found
best_params = rf_random.best_params_
rf_grid_params = {
            'n_estimators': [best_params['n_estimators'] - 50, best_params['n_estimators'], best_params['n_estimators'] + 50],
            'max_depth': [best_params['max_depth'] - 10, best_params['max_depth'], best_params['max_depth'] + 10],            
            'criterion': [best_params['criterion']],
            'bootstrap': [best_params['bootstrap']]
}
rscv = GridSearchCV(rf_classifier, rf_grid_params, cv=5, scoring='f1', n_jobs=-1)
rscv.fit(x_train_tfidf, y_train)
print("Refined best parameters from GridSearchCV: ", rscv.best_params_)


# Fitting 3 folds for each of 100 candidates, totalling 300 fits
# Best parameters found by RandomizedSearchCV for SVM:
# {'kernel': 'rbf', 'gamma': 0.001, 'C': 545.5594781168514}
# Fitting 5 folds for each of 9 candidates, totalling 45 fits
# Refined best parameters from GridSearchCV for SVM:
# {'C': 272.7797390584257, 'gamma': 0.002, 'kernel': 'rbf'}

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best parameters found by RandomizedSearchCV for SVM:
{'kernel': 'rbf', 'gamma': 0.001, 'C': 545.5594781168514}
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Refined best parameters from GridSearchCV for SVM:
{'C': 272.7797390584257, 'gamma': 0.002, 'kernel': 'rbf'}
Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [71]:
from sklearn.metrics import classification_report

classifiers = {
    "Random Forest": RandomForestClassifier(max_depth=80, n_estimators=950, criterion='gini', bootstrap=False),
    "Support Vector Classification": SVC(gamma='scale', kernel='linear', C=1.99),
    "Decision Tree": DecisionTreeClassifier(max_depth=75, criterion='gini'),
    "Logistic Regression": LogisticRegression(penalty='l1', C=1.0, solver='liblinear'),
    "Multinomial Naive Bayes": MultinomialNB(alpha=2.63),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=69, weights='uniform')
}

evaluation_reports = {}

# Train, predict, and evaluate each classifier separately
for classifier_name, classifier in classifiers.items():
    print(f"Training {classifier_name}...")
    
    # Train the classifier
    classifier.fit(x_train_tfidf, y_train)
    
    # Predict on the test set
    y_pred = classifier.predict(x_test_tfidf)
    
    # Generate a classification report
    report = classification_report(y_test, y_pred, output_dict=True)
    
    # Print accuracy and store the report for further analysis
    print(f"{classifier_name} Accuracy: {report['accuracy']}")
    evaluation_reports[classifier_name] = report
    
    # Detailed report
    print(f"{classifier_name} Classification Report:")
    print(classification_report(y_test, y_pred))
    


Training Random Forest...


KeyboardInterrupt: 